## Others things

In [ ]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

import pyten

In [ ]:
if imputationType == "LRTC":
    percentage_nans = (df_filter.isnull().sum().sum() / (df_filter.shape[0]*df_filter.shape[1]) 
    maxRank = int(np.round((X.shape[1] * X.shape[2]) * percentage_nans))

    n1, n2, n3 = Xf.shape
    coords = np.array(np.meshgrid(range(n1), range(n2), range(n3))).reshape(3, -1).T
    df_lrtc = pd.DataFrame(coords, columns=["x1", "x2", "x3"])
    df_lrtc["r"] = Xf[coords[:, 0], coords[:, 1], coords[:, 2]]
    df_lrtc["x1"] += 1
    df_lrtc["x2"] += 1
    df_lrtc["x3"] += 1

    rank = 5
    [OriTensor, DeTensor, TenClass, RecTensor, RecTensor_hat, mask] = pyten.UI.helios(df_lrtc, rank)

    # Calculate the error between the real tensor and the reconstructed tensor
    # we only consider the values different from nan
    rec = RecTensor_hat * mask 
    real = np.nan_to_num(Xf)
    error = np.linalg.norm(rec - real)/np.linalg.norm(real)
    print(error)
    fin = time.time()

    finalTime_LRTC = fin - inicio
    print(finalTime_LRTC)

    X_LRTC = RecTensor_hat 

# Other things 2

In [ ]:
def timedelta_to_hours(td):
    if pd.isnull(td):
        return np.nan
    else:
        return td.total_seconds() / 3600

def getNans(df):

    pat_uniques = df.patientunitstayid.unique()
    keys = list(df.keys())[2:]

    nans_per_pat = []
    for p in range(len(pat_uniques)):
        nans_per_feature = []
        for f in range(len(keys)):
            pat = df[df.patientunitstayid == pat_uniques[p]]
            nan_values = pat[[keys[f]]].isnull().sum()
            nan_values = nan_values.values[0]
            nans_per_feature.append(np.round(nan_values/pat.shape[0], 5) * 100)

        nans_per_pat.append(nans_per_feature)

    df_nans = pd.DataFrame(nans_per_pat, columns=df.columns[2:])
    df_nans.insert(loc=0, column='patientunitstayid', value=df.patientunitstayid.unique())

    return df_nans


def temporalDataset(df_grupo, maxLength, flag):
    
    if len(df_grupo) < maxLength:
        fill_values = {col: [flag] * (maxLength - len(df_grupo)) for col in df_grupo.columns[1:]}  
        df_grupo = df_grupo.append(pd.DataFrame({'patientunitstayid': [df_grupo['patientunitstayid'].iloc[0]] * (maxLength - len(df_grupo)), **fill_values}), ignore_index=True)
    else: 
        df_grupo = df_grupo.head(maxLength)
        
    return df_grupo


def temporalDatasetRemovingPatients(df_grupo, maxLength, flag):
    if len(df_grupo) < maxLength:
        return None
    else:
        return df_grupo.head(maxLength)

## Step 1: Filter per # of time steps

In [ ]:
print("# of patients:", len(df.patientunitstayid.unique()))
print("# of total rows:", len(df.patientunitstayid.unique()) * numberOfTimeSteps)
df_icu_entry = df[df.infusionoffset >= 0]
# df_temp = df_icu_entry.groupby('patientunitstayid').apply(utils.temporalDataset, numberOfTimeSteps, flag).reset_index(drop=True)
df_temp = df_icu_entry.groupby('patientunitstayid').apply(utils.temporalDatasetRemovingPatients, numberOfTimeSteps, flag).reset_index(drop=True)
# df_nans_temp = utils.getNans(df_temp)
# df_nans_temp

## Step 2. Matrix to tensor

In [ ]:
X = dataframeToTensor(df_temp, numberOfTimeSteps)
print(X.shape)

## Step 3.A. LRTC imputation

In [ ]:
inicio = time.time()


n1, n2, n3 = Xf.shape
coords = np.array(np.meshgrid(range(n1), range(n2), range(n3))).reshape(3, -1).T
df_lrtc = pd.DataFrame(coords, columns=["x1", "x2", "x3"])
df_lrtc["r"] = Xf[coords[:, 0], coords[:, 1], coords[:, 2]]
df_lrtc["x1"] += 1
df_lrtc["x2"] += 1
df_lrtc["x3"] += 1

rank = 5
[OriTensor, DeTensor, TenClass, RecTensor, RecTensor_hat, mask] = pyten.UI.helios(df_lrtc, rank)

# Calculate the error between the real tensor and the reconstructed tensor
# we only consider the values different from nan
rec = RecTensor_hat * mask 
real = np.nan_to_num(Xf)
error = np.linalg.norm(rec - real)/np.linalg.norm(real)
print(error)
fin = time.time()

finalTime_LRTC = fin - inicio
print(finalTime_LRTC)

X_LRTC = RecTensor_hat